In [1]:
import pickle
import tensorflow as tf
import tensorflow.keras
import numpy as np
import pandas as pd
from utils import vgg_train_dataset, vgg_train_dataset_ps

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='1' # '0' for DEBUG=all [default], '1' to filter INFO msgs, '2' to filter WARNING msgs, '3' to filter all msgs

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, BatchNormalization,Activation,Reshape, Permute, GRU, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

if tf.test.gpu_device_name():
    print('Default GPU Device Details: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install Tensorflow that supports GPU")

physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

if not os.path.exists('models/vgg'):
    os.makedirs('models/vgg')
if not os.path.exists('hist/vgg'):
    os.makedirs('hist/vgg')

Default GPU Device Details: /device:GPU:0


In [4]:
num_classes=8
# X,Y =vgg_train_dataset()
X,Y =vgg_train_dataset_ps()
print(X.shape, Y.shape)

(11985, 31, 128) (11985,)
(11985, 31, 128, 1) (11985, 8)


In [3]:
X_train, X_validation, y_train, y_validation = train_test_split(X, Y, train_size=0.8, random_state=1234)

In [4]:
def mlp(batch_size,epochs):
    model = Sequential()
    model.add(Flatten(input_shape = (X.shape[1],X.shape[2],1)))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(num_classes,activation='softmax'))
    
    earlyStopping = EarlyStopping(monitor='val_loss', patience=25, verbose=0, mode='min')
    checkpoint = ModelCheckpoint('./models/vgg/model-{epoch:03d}-{val_loss:03f}-{val_accuracy:03f}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='min')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    hist = model.fit(X_train,y_train,validation_data=(X_validation,y_validation),batch_size=batch_size,epochs=epochs,callbacks=[earlyStopping,checkpoint])
    return model

In [5]:
def crnn(batch_size,epochs):
    model = Sequential()
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',input_shape=(X.shape[1],X.shape[2],1),activation='elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.5))
    model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',activation='elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(4,4),padding='same'))
    model.add(Dropout(0.5))

    model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',activation='elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(4,4),padding='same'))
    model.add(Dropout(0.5))
    model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',activation='elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(5,5),padding='same'))
    model.add(Dropout(0.5))
    model.add(Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(5,5),padding='same'))
    model.add(Dropout(0.5))

    resize_shape = model.output_shape[2] * model.output_shape[3]
    model.add(Reshape((model.output_shape[1], resize_shape)))

    # recurrent layer
    model.add(GRU(64, return_sequences=True))
    model.add(GRU(64, return_sequences=False))
    model.add(Dropout(0.3))
    model.add(Dense(num_classes,activation='softmax'))
    
    earlyStopping = EarlyStopping(monitor='val_accuracy', patience=25, verbose=0, mode='max')
    checkpoint = ModelCheckpoint('./models/vgg/model-{epoch:03d}-{val_loss:03f}-{val_accuracy:03f}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='min')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    hist = model.fit(X_train,y_train,validation_data=(X_validation,y_validation),batch_size=batch_size,epochs=epochs,callbacks=[earlyStopping,checkpoint])
    np.save("hist/vgg/hist.npy",hist.history)
    return model

In [6]:
def cnn(batch_size,epochs):
    model = Sequential()
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding='same',input_shape=(X.shape[1],X.shape[2],1),activation='elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=128,kernel_size=(3,3),padding='same',activation='elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.2))

    model.add(Conv2D(filters=256,kernel_size=(3,3),padding='same',activation='elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.2))
    model.add(Conv2D(filters=512,kernel_size=(3,3),padding='same',activation='elu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(num_classes,activation='softmax'))
    
    earlyStopping = EarlyStopping(monitor='val_loss', patience=25, verbose=0, mode='min')
    checkpoint = ModelCheckpoint('./models/vgg/model-{epoch:03d}-{val_loss:03f}-{val_accuracy:03f}.h5', verbose=1, monitor='val_loss',save_best_only=True, mode='min')
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    hist = model.fit(X_train,y_train,validation_data=(X_validation,y_validation),batch_size=batch_size,epochs=epochs,callbacks=[earlyStopping,checkpoint])
#     np.save("hist/vgg/hist.npy",hist.history)
    return model

In [7]:
# crnn(32,400)
# cnn(32,400)
mlp(32,400)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 3968)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                127008    
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 1032      
Total params: 138,472
Trainable params: 138,472
Non-trainable params: 0
_________________________________________________________________
Epoch 1/400
288/300 [===========================>..] - ETA: 0s - loss: 1.1882 - accuracy: 0.5897
Epoch 00001: val_lo